In [1]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 2.7.13 |Anaconda 4.3.1 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
scipy: 0.18.1
numpy: 1.11.3
matplotlib: 2.0.0
pandas: 0.19.2
sklearn: 0.18.1


In [2]:
# Load libraries
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVC

In [3]:
import datetime
from datetime import date,time
import time

In [4]:
pd.options.mode.chained_assignment = None 

In [5]:
#def time_num(date):
#    intervalo = 30
    
#    hora = date.hour
#    minuto = date.minute
    
#    num_minutes = hora*60 + minuto
    
#    time_bin = num_minutes / intervalo
#    hour_bin = num_minutes / 60
#    min_bin = (time_bin * intervalo) % 60
    
#    return ((hour_bin * 60 + min_bin + intervalo) / 2.0)/(60*24)

In [6]:
dataset = pd.read_csv('data/trip_train.csv')
weather = pd.read_csv('data/weather.csv')
station = pd.read_csv('data/station.csv')

In [7]:
station = station.loc[:,['id','lat','long']]

In [8]:
weather.date = pd.to_datetime(weather.date)

In [9]:
weather.precipitation_inches = weather.precipitation_inches.apply(lambda x: 0 if x == 'T' else x)

In [10]:
weather = weather.loc[:,['date','mean_temperature_f','mean_humidity']].groupby('date').mean()
weather = pd.DataFrame(weather.to_records())

In [11]:
weather

,date,mean_temperature_f,mean_humidity
0,2013-08-29,70.4,73.20
1,2013-08-30,73.0,68.80
2,2013-08-31,68.0,70.80
3,2013-09-01,70.0,66.80
4,2013-09-02,70.8,74.00
5,2013-09-03,68.8,64.20
6,2013-09-04,68.6,67.20
7,2013-09-05,68.0,65.80
8,2013-09-06,72.0,57.20
9,2013-09-07,76.2,52.80


In [12]:
dataset = dataset.loc[(dataset.duration < 86400) & (dataset.duration > 60),:]

In [64]:
prueba = dataset.head(80000)
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,460948.162287,980.402925,57.934762,57.931762,427.984100
std,264197.852679,3188.269060,17.035207,17.120553,153.508376
min,4073.000000,61.000000,2.000000,2.000000,9.000000
25%,231909.000000,345.000000,50.000000,50.000000,334.000000
50%,458768.000000,518.000000,62.000000,62.000000,442.000000
75%,692696.250000,757.000000,70.000000,70.000000,546.000000
max,913429.000000,85840.000000,84.000000,84.000000,878.000000


In [14]:
prueba.start_date = pd.to_datetime(prueba.start_date)

In [15]:
prueba['date'] = prueba.start_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))

In [16]:
def devolver_estacion (fecha):
    if (fecha >= datetime.datetime(fecha.year,3,21)) & (fecha < datetime.datetime(fecha.year,6,21)):
        return 'Primavera'
    if (fecha >= datetime.datetime(fecha.year,6,21)) & (fecha < datetime.datetime(fecha.year,9,21)):
        return 'Verano'
    if (fecha >= datetime.datetime(fecha.year,9,21)) & (fecha < datetime.datetime(fecha.year,12,21)):
        return 'Otonio'
    else:
        return 'Invierno'

In [17]:
prueba['estacion'] = prueba.date.apply(lambda x: devolver_estacion(x))

In [18]:
prueba = pd.merge(prueba,weather,on='date',how='inner')

In [19]:
weather

,date,mean_temperature_f,mean_humidity
0,2013-08-29,70.4,73.20
1,2013-08-30,73.0,68.80
2,2013-08-31,68.0,70.80
3,2013-09-01,70.0,66.80
4,2013-09-02,70.8,74.00
5,2013-09-03,68.8,64.20
6,2013-09-04,68.6,67.20
7,2013-09-05,68.0,65.80
8,2013-09-06,72.0,57.20
9,2013-09-07,76.2,52.80


In [20]:
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id,mean_temperature_f,mean_humidity
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000
mean,460428.359182,985.926872,57.866485,57.862005,427.635018,62.135443,66.914513
std,264685.723929,3254.075288,17.089099,17.177565,153.712385,6.761382,9.868337
min,4069.000000,61.000000,2.000000,2.000000,9.000000,39.200000,31.200000
25%,231089.750000,344.000000,50.000000,50.000000,334.000000,57.400000,62.600000
50%,459252.500000,517.000000,62.000000,62.000000,441.000000,62.000000,68.400000
75%,692916.250000,755.000000,70.000000,70.000000,546.000000,67.800000,72.400000
max,913459.000000,86363.000000,84.000000,84.000000,878.000000,80.400000,92.500000


In [21]:
prueba['hour_start'] = prueba.start_date.apply(lambda x: x.hour)
prueba['weekend'] = prueba.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)

In [22]:
prueba.subscription_type = prueba.subscription_type.apply(lambda x: 1 if x == 'Customer' else 0)

In [23]:
prueba = prueba.loc[:,['duration','hour_start','weekend','estacion','subscription_type']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549678 entries, 0 to 549960
Data columns (total 11 columns):
id                    549678 non-null int64
duration              549678 non-null int64
start_date            549678 non-null object
start_station_name    549678 non-null object
start_station_id      549678 non-null int64
end_date              549678 non-null object
end_station_name      549678 non-null object
end_station_id        549678 non-null int64
bike_id               549678 non-null int64
subscription_type     549678 non-null object
zip_code              544300 non-null object
dtypes: int64(5), object(6)
memory usage: 50.3+ MB


In [24]:
prueba

,duration,hour_start,weekend,estacion,subscription_type
0,396,8,0,Verano,0
1,486,12,0,Verano,0
2,1027,8,0,Verano,0
3,354,8,0,Verano,0
4,209,15,0,Verano,0
5,849,8,0,Verano,0
6,981,9,0,Verano,0
7,239,13,0,Verano,0
8,698,20,0,Verano,1
9,414,17,0,Verano,0


In [25]:
prueba['hora_pico'] = prueba.hour_start.apply(lambda x: 1 if (((x>=0) & (x<=3) ) | ((x>=10) & (x<=16))) else 0) 

In [26]:
estaciones = pd.get_dummies(prueba['estacion'])
prueba = prueba.join(estaciones)
prueba = prueba.drop('estacion',axis=1)
prueba = prueba.drop('hour_start',axis=1)
#prueba

In [27]:
#start_station_name = pd.get_dummies(prueba['start_station_name'])
#end_station_name = pd.get_dummies(prueba['end_station_name'])

In [28]:
#print start_station_name.shape
#print end_station_name.shape

In [29]:
#prueba = prueba.drop('start_station_name',axis=1)
#prueba = prueba.drop('end_station_name',axis=1)
#prueba.shape

In [30]:
#prueba = prueba.join(start_station_name)

In [31]:
#columnas = [x +  '_end' for x in list(end_station_name.columns.tolist())]
#end_station_name.columns = columnas

In [32]:
#prueba = prueba.join(end_station_name)

In [33]:
#days = pd.get_dummies(prueba['weekday'])
#prueba = prueba.join(days)
#prueba = prueba.drop('weekday',axis=1)
#prueba

In [34]:
prueba

,duration,weekend,subscription_type,hora_pico,Invierno,Otonio,Primavera,Verano
0,396,0,0,0,0,0,0,1
1,486,0,0,1,0,0,0,1
2,1027,0,0,0,0,0,0,1
3,354,0,0,0,0,0,0,1
4,209,0,0,1,0,0,0,1
5,849,0,0,0,0,0,0,1
6,981,0,0,0,0,0,0,1
7,239,0,0,1,0,0,0,1
8,698,0,1,0,0,0,0,1
9,414,0,0,0,0,0,0,1


In [35]:
columnas = prueba.columns.tolist()
columnas = columnas[1:len(columnas)]
columnas.append('duration')
prueba = prueba[columnas]
len(prueba.columns.tolist())

8

In [36]:
from sklearn import preprocessing

In [37]:
array = prueba.values
X = array[:,0:len(prueba.columns.tolist())-1]
Y = array[:,len(prueba.columns.tolist())-1]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Pruebo los distintos algoritmos

In [ ]:
start_time = time.time()
models = []
#models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('RDMF',RandomForestRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('SGD',SGDRegressor()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
for name, model in models:
    model.fit(X_train,Y_train)
    predictions = model.predict(X_validation)
    msg = "%s -> Mean Squared Error: %f" % (name, mean_squared_error(Y_validation, predictions))
    print(msg)
print("--- %s seconds ---" % (time.time() - start_time))

##### EN GENERAL LOS MEJORES SON KNN Y RANDOM FORESTS

# Pruebo KNN con distintos Ks

In [ ]:
start_time = time.time()
for k in range(1,11):
    lda = KNeighborsClassifier(n_neighbors = k)
    lda.fit(X_train, Y_train)
    predictions = lda.predict(X_validation)
    print "Con K = %i: %f" % (k,mean_squared_error(Y_validation, predictions))    
print("--- %s seconds ---" % (time.time() - start_time))

# Pruebo RandomForests con distinto # de arboles

In [40]:
start_time = time.time()
for k in range(10,20):
    rdf = RandomForestRegressor(n_estimators = k)
    rdf.fit(X_train, Y_train)
    predictions = rdf.predict(X_validation)
    print "Con %i arboles: %f" % (k,mean_squared_error(Y_validation, predictions))    
print("--- %s seconds ---" % (time.time() - start_time))

Con K = 11: 9464382.681237
Con K = 13: 9465153.258834
Con K = 15: 9465149.395720
Con K = 17: 9464541.755128
Con K = 19: 9466044.526802
Con K = 21: 9464550.895609
Con K = 23: 9464528.579044
Con K = 25: 9465672.333988
Con K = 27: 9465034.685561
Con K = 29: 9464352.312867
--- 78.0929210186 seconds ---


# ----------------------------------------------------------------------------------------------------

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
#scaler_train = MinMaxScaler(feature_range=(0, 1))
#rescaledX_train = scaler_train.fit_transform(X_train)

#scaler_validation = MinMaxScaler(feature_range=(0, 1))
#rescaledX_validation = scaler_validation.fit_transform(X_validation)

#normalknn = LogisticRegression()
#normalknn.fit(rescaledX_train, Y_train)
#predictions = normalknn.predict(rescaledX_validation)

#print mean_squared_error(Y_validation, predictions)

In [41]:
test = pd.read_csv('data/trip_test.csv')

In [42]:
test.start_date = pd.to_datetime(test.start_date)

In [43]:
test['date'] = test.start_date.apply(lambda x: datetime.datetime(x.year,x.month,x.day,0,0))

In [44]:
test['weekend'] = test.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)

In [45]:
test.subscription_type = test.subscription_type.apply(lambda x: 1 if x == 'Customer' else 0)

In [46]:
test['hour_start'] = test.start_date.apply(lambda x: x.hour)

In [47]:
test['estacion'] = test.date.apply(lambda x: devolver_estacion(x))

In [48]:
test['hora_pico'] = test.hour_start.apply(lambda x: 1 if (((x>=0) & (x<=3) ) | ((x>=10) & (x<=16))) else 0)

In [49]:
estaciones = pd.get_dummies(test['estacion'])
test = test.join(estaciones)
test = test.drop('estacion',axis=1)
test = test.drop('hour_start',axis=1)

In [50]:
test = test.loc[:,['weekend','subscription_type','hora_pico','Invierno','Otonio','Primavera','Verano']]

In [51]:
test

,weekend,subscription_type,hora_pico,Invierno,Otonio,Primavera,Verano
0,1,1,1,0,1,0,0
1,0,0,1,0,1,0,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0
5,0,0,0,0,1,0,0
6,0,0,0,1,0,0,0
7,0,0,0,0,0,1,0
8,1,0,1,0,0,0,1
9,0,0,1,0,0,0,1


In [52]:
#test = test.drop('start_date',axis=1)

In [53]:
#test = pd.merge(test,weather,on='date',how='inner')

In [54]:
#test = test.drop('date',axis=1)

In [55]:
#test = test[['mean_temperature_f','time_num','weekend']]

In [56]:
array_test = test.values

In [57]:
rfr = RandomForestRegressor(n_estimators = 11)
rfr.fit(X_train, Y_train)
predictions = rfr.predict(array_test)

In [58]:
predictions = pd.DataFrame(predictions)

In [59]:
predictions.columns = ['duration']
predictions

,duration
0,3620.427241
1,577.340036
2,556.156810
3,566.605255
4,556.156810
5,587.866564
6,548.332336
7,556.156810
8,659.784001
9,580.854538


In [60]:
test = pd.read_csv('data/trip_test.csv')

In [61]:
test = test.join(predictions)

In [62]:
test = test.loc[:,['id','duration']]

In [63]:
test.to_csv('submission4.csv',index=False)